# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=21

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==21]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm21', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm21/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-07 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_23140/3484638029.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_23140/3484638029.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_23140/3484638029.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.282307170 0.9319660 -2.015896918  12.64717236
       -0.278218281 0.9258477 -1.978804088  12.57852033
       -0.274146043 0.9198393 -1.941880811  12.50950668
       -0.270090321 0.9139404 -1.905129128  12.44013783
       -0.266050981 0.9081505 -1.868550994  12.37041972
       -0.262027892 0.9024693 -1.832148281  12.30035783
       -0.258020924 0.8968962 -1.795922782  12.22995715
       -0.254029947 0.8914309 -1.759876215  12.15922226
       -0.250054835 0.8860728 -1.724010229  12.08815728
       -0.246095461 0.8808215 -1.688326404  12.01676589
       -0.242151703 0.8756765 -1.652826257  11.94505136
       -0.238223437 0.8706373 -1.617511249  11.87301653
       -0.234310541 0.8657034 -1.582382781  11.80066383
       -0.230412897 0.8608743 -1.547442204  11.72799532
       -0.226530386 0.8561495 -1.512690819  11.65501263
       -0.22266

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]        [,4]
sigmas -0.612511715 1.3517832 -2.196874390  2.27437808
       -0.602078535 1.3394620 -2.186914067  2.31892451
       -0.591753083 1.3271835 -2.176617064  2.36299388
       -0.581533157 1.3149500 -2.165985576  2.40657648
       -0.571416623 1.3027637 -2.155021908  2.44966264
       -0.561401408 1.2906267 -2.143728478  2.49224275
       -0.551485504 1.2785413 -2.132107818  2.53430726
       -0.541666961 1.2665095 -2.120162574  2.57584671
       -0.531943885 1.2545336 -2.107895507  2.61685172
       -0.522314438 1.2426158 -2.095309492  2.65731302
       -0.512776833 1.2307581 -2.082407523  2.69722144
       -0.503329335 1.2189628 -2.069192706  2.73656793
       -0.493970258 1.2072319 -2.055668266  2.77534359
       -0.484697962 1.1955676 -2.041837539  2.81353964
       -0.475510851 1.1839720 -2.027703980  2.85114747
       -0.466407376 1.1724472 -2.013271156  2.88815863
       -0.457386028 1.1609952 -

        0.320323779 0.6619637  0.547229651  2.03292881
        0.324441571 0.6652579  0.558067749  1.99092889
        0.328542477 0.6686111  0.568620870  1.94851228
        0.332626634 0.6720212  0.578886071  1.90567943
        0.336694178 0.6754864  0.588860402  1.86243072
        0.340745244 0.6790051  0.598540907  1.81876651
        0.344779966 0.6825752  0.607924622  1.77468709
        0.348798474 0.6861948  0.617008573  1.73019271
        0.352800898 0.6898622  0.625789782  1.68528359
        0.356787366 0.6935753  0.634265259  1.63995990
        0.360758006 0.6973322  0.642432009  1.59422177
        0.364712942 0.7011309  0.650287026  1.54806932
        0.368652298 0.7049695  0.657827298  1.50150262
        0.372576197 0.7088459  0.665049803  1.45452173
        0.376484758 0.7127581  0.671951513  1.40712669
        0.380378102 0.7167040  0.678529390  1.35931750
        0.384256347 0.7206817  0.684780389  1.31109416
        0.388119609 0.7246888  0.690701458  1.26245667
        0.

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_23140/3484638029.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_23140/3484638029.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_23140/3484638029.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]          [,3]     [,4]
sigmas -0.5960867773 1.2469884 -2.273955e+00 3.684390
       -0.5872844814 1.2359069 -2.255436e+00 3.681686
       -0.5785589902 1.2249184 -2.236953e+00 3.679125
       -0.5699089752 1.2140227 -2.218504e+00 3.676711
       -0.5613331418 1.2032195 -2.200088e+00 3.674445
       -0.5528302283 1.1925088 -2.181705e+00 3.672330
       -0.5443990052 1.1818901 -2.163355e+00 3.670368
       -0.5360382738 1.1713633 -2.145036e+00 3.668559
       -0.5277468649 1.1609282 -2.126748e+00 3.666905
       -0.5195236386 1.1505845 -2.108489e+00 3.665408
       -0.5113674826 1.1403319 -2.090259e+00 3.664069
       -0.5032773117 1.1301704 -2.072056e+00 3.662887
       -0.4952520667 1.1200995 -2.053881e+00 3.661864
       -0.4872907138 1.1101192 -2.035731e+00 3.661001
       -0.4793922438 1.1002292 -2.017606e+00 3.660296
       -0.4715556711 1.0904292 -1.999504e+00 3.

        0.2443294720 0.5836140  4.188542e-01 3.188835
        0.2481374416 0.5857732  4.342283e-01 3.168565
        0.2519309655 0.5880071  4.494743e-01 3.147940
        0.2557101530 0.5903148  4.645898e-01 3.126961
        0.2594751120 0.5926956  4.795724e-01 3.105627
        0.2632259493 0.5951488  4.944194e-01 3.083938
        0.2669627703 0.5976735  5.091286e-01 3.061892
        0.2706856794 0.6002690  5.236973e-01 3.039489
        0.2743947799 0.6029343  5.381232e-01 3.016729
        0.2780901737 0.6056687  5.524038e-01 2.993612
        0.2817719620 0.6084713  5.665366e-01 2.970136
        0.2854402443 0.6113413  5.805191e-01 2.946302
        0.2890951195 0.6142777  5.943489e-01 2.922109
        0.2927366853 0.6172796  6.080235e-01 2.897557
        0.2963650381 0.6203462  6.215403e-01 2.872645
        0.2999802736 0.6234765  6.348968e-01 2.847372
        0.3035824862 0.6266695  6.480907e-01 2.821739
        0.3071717694 0.6299244  6.611194e-01 2.795745
        0.3107482158 0.63324

In [2]:
print(error_i)

[1, 2, 3, 4, 9, 10, 11, 12, 14, 17]
